In [14]:
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

class GithubApis:
    def __init__(self):
        # GitHub username and repository
        self.username = "neonwatty"
        self.repository = "meme-search"

        # Load the personal access token from .env
        self.access_token = os.getenv("GITHUB_TOKEN")
        if not self.access_token:
            raise ValueError("GitHub token not found. Please set GITHUB_TOKEN in the .env file.")

        # Form headers with authentication
        self.headers = {
            "Authorization": f"token {self.access_token}",
            "Accept": "application/vnd.github+json"
        }
        
    def get_data(self, endpoint):
        # GitHub API URL for endpoint
        url = f"https://api.github.com/repos/{self.username}/{self.repository}/{endpoint}"

        # Make the GET request
        response = requests.get(url, headers=self.headers)

        # Check the response
        if response.status_code == 200:
            self.traffic_data = response.json()
            return self.traffic_data
        else:
            print(f"Failed to fetch data for endpoint {endpoint} - received: {response.status_code}")
            print(response.json())

In [21]:
# endpoint for views
endpoint = 'traffic/views'

# endpoint for clones
endpoint = 'traffic/clones'

# endpoint for top referral references
endpoint = 'traffic/popular/referrers'

# endpoint for popular paths
endpoint = 'traffic/popular/paths'

data_fetcher = GithubApis()
data = data_fetcher.get_data(endpoint)

In [22]:
data

[{'path': '/neonwatty/meme-search',
  'title': 'neonwatty/meme-search: The open source Meme Search Engine and Finder. Free an...',
  'count': 69,
  'uniques': 46},
 {'path': '/neonwatty/meme-search/graphs/traffic',
  'title': 'Traffic · neonwatty/meme-search',
  'count': 5,
  'uniques': 1},
 {'path': '/neonwatty/meme-search/tree/landing-page',
  'title': 'neonwatty/meme-search at landing-page',
  'count': 5,
  'uniques': 1},
 {'path': '/neonwatty/meme-search/blob/main/docker-compose-pro.yml',
  'title': 'meme-search/docker-compose-pro.yml at main · neonwatty/meme-search',
  'count': 4,
  'uniques': 1},
 {'path': '/neonwatty/meme-search/tree/main/meme_search',
  'title': 'meme-search/meme_search at main · neonwatty/meme-search · GitHub',
  'count': 3,
  'uniques': 2},
 {'path': '/neonwatty/meme-search/blob/landing-page/index.html',
  'title': 'meme-search/index.html at landing-page · neonwatty/meme-search',
  'count': 3,
  'uniques': 1},
 {'path': '/neonwatty/meme-search/pulse',
  'titl

In [7]:
sum([v['uniques'] for v in data['views']])

64